In [ ]:
import torch_geometric as tg
import torch
import cugraph as cg

In [ ]:
graph = torch.load('data/Knn-G/tgGraph_2000h_4k.pt',weights_only=False)
graph

Data(x=[100000, 5], edge_index=[2, 10000000], edge_attr=[10000000])

In [ ]:
G = tg.utils.to_cugraph(graph.edge_index, graph.edge_attr, directed=False)
G

In [ ]:
pos_df = cg.force_atlas2(
    G, 
    max_iter=100,           # Increase this if the graph still looks "active"
    scaling_ratio=20.0,      # Higher = more spread out (try 10.0 to 100.0)
    gravity=1.0,             # Prevents nodes from drifting to infinity
    strong_gravity_mode=True, 
    jitter_tolerance=1.0     # Helps nodes settle faster
)
pos_df

,vertex,x,y
0,46473,13.646615,-756.606750
1,8653,-2209.288086,-275.507935
2,44472,1248.729980,-1860.306396
3,49673,55.157990,-766.600769
4,27662,155.812363,-2217.541260
...,...,...,...
99995,99989,-1680.625366,609.607117
99996,99992,-1540.484009,-725.821899
99997,99993,-1541.569458,-727.563721
99998,99994,-1541.152222,-726.282532


In [52]:
nodes_df = pos_df.to_pandas()

theta_np = graph.x.cpu().numpy()
theta_strings = [",".join(map(str, row)) for row in theta_np]
nodes_df['features'] = theta_strings

nodes_df.to_csv('data/Knn-G-Eval/cosmo_nodes.csv', index=False)

In [53]:
edges_gdf = G.view_edge_list()
edges_df = edges_gdf.to_pandas()
edges_df.columns = ['source', 'target', 'weight']

edges_df.to_csv('data/Knn-G-Eval/cosmo_edges.csv', index=False)